In [212]:
#Python 3.11.2
#Import packages
import pandas as pd
import numpy as np
import pylab as pl
import random
import matplotlib.pyplot as plt
from collections import defaultdict
import json # for pretty printing
import geopandas as gpd
import os

We have three different data sources. 

1. The data collected by Bodil corresponds to the plague period.
2. The information from the TABVERK database includes the population size for parishes in the posterior years of the plague.
3. The geographical information (polygons) for some parishes. This information doesn't correspond to the plague period.

Our goal is to create a unique database for our project: Plague spread across Scania, Sweden, from 1710 to 1715.

In [213]:
# For public files paths
data_folder = "data"
appendix6_path = os.path.join(data_folder, "Appendix6Bodil.csv")

# For private files paths
data_private_folder = "data/private"
allParishes_path = os.path.join(data_private_folder, "allParishesScania.xlsx")

First we read the different data sources (.xlsx, .csv, and .shp files)

In [214]:
# Bodil's data Appendix 6 plague parishes
plagueParishesScania = pd.read_csv(appendix6_path, sep=",", encoding="utf-8")
# All parishes in Scania during the plague period
allParishesScania = pd.read_excel(allParishes_path)

Transforming the lowercase to uppercase and checking the type

In [215]:
allParishesScania = allParishesScania.apply(
    lambda x: x.astype(str).str.upper())
plagueParishesScania = plagueParishesScania.apply(
    lambda x: x.astype(str).str.upper())
type(plagueParishesScania)
type(allParishesScania)

pandas.core.frame.DataFrame

Visualizing the DataFrames

In [216]:
len(plagueParishesScania)
#allParishesScania.head(3)

178

In [217]:
len(allParishesScania)
#plagueParishesScania.head(3)

398

Merging the two datasets (allParishesScania and plagueParishesScania)

In [218]:
parishesScania = pd.merge(
    allParishesScania, plagueParishesScania, how='left', on=['ParishName', 'Region'])

Checking that the new data frame keep all the outbreaks for parish

In [219]:
parishesScania.loc[parishesScania['ParishName'] == 'NÄSUM']

,Region,District(Härad),ParishName,BeginPlaguePeriod,EndPlaguePeriod,VictimsNumber
393,NORTHEAST,VILLANDS,NÄSUM,NOV 1710,APR 1711,671
394,NORTHEAST,VILLANDS,NÄSUM,FEB 1712,UNDEFINED,?


Defining a function for extracting the names of the parishes in the data frame

In [220]:
def get_Names(data: pd.DataFrame, heading:str) -> list:
    return data[heading]   

In [221]:
parishesScania_names = get_Names(parishesScania, 'ParishName').unique().tolist()
len(parishesScania_names)

397

The length of 'parishesScania_names' is less than the number of rows in the data frame 'allparishesScania'. This means, there is a repeated name: 'LÖDDEKÖPINGE'. We have to check the information for this parish:

In [222]:
parishesScania.loc[parishesScania['ParishName'] == 'LÖDDEKÖPINGE']

,Region,District(Härad),ParishName,BeginPlaguePeriod,EndPlaguePeriod,VictimsNumber
87,SOUTHWEST,HARJAGER,LÖDDEKÖPINGE,AUG 1712,DEC 1712,?
161,SOUTHWEST,TORNA,LÖDDEKÖPINGE,AUG 1712,DEC 1712,?


Only the parish LÖDDEKÖPINGE at HARJAGER was affected by the plague according to the file 'Bilaga 6 d - sydväst.doc' provided by Bodil. So we need to fix the information in the other row (161).

In [223]:
parishesScania.at[161, 'BeginPlaguePeriod'] = np.NaN
parishesScania.at[161, 'EndPlaguePeriod'] = np.NaN
parishesScania.at[161, 'VictimsNumber'] = np.NaN

Checking the data:

In [224]:
parishesScania.loc[parishesScania['ParishName'] == 'LÖDDEKÖPINGE']

,Region,District(Härad),ParishName,BeginPlaguePeriod,EndPlaguePeriod,VictimsNumber
87,SOUTHWEST,HARJAGER,LÖDDEKÖPINGE,AUG 1712,DEC 1712,?
161,SOUTHWEST,TORNA,LÖDDEKÖPINGE,NaN,NaN,NaN


Filtering the data frame by region and then get the names of the parishes:

In [225]:
southeastParishes = allParishesScania.loc[allParishesScania['Region'] == 'SOUTHEAST']
southeastParishes_names = get_Names(southeastParishes, 'ParishName')

Reading the census file:

In [226]:
# Set the working directory for private files
data_private_folder = "data/private"
census_path = os.path.join(data_private_folder, 'FILE01_FALD.csv')
censusSweden = pd.read_csv(census_path, sep=';')
censusSweden.shape

(102360, 50)

Checking the memory usage

In [227]:
censusSweden.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102360 entries, 0 to 102359
Data columns (total 50 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LANGEN       102360 non-null  int64  
 1   LANGENNMN    102360 non-null  object 
 2   GEOID        102360 non-null  int64  
 3   GEOIDNMN     102360 non-null  object 
 4   GEOIDTYP     102360 non-null  int64  
 5   AR           102360 non-null  int64  
 6   FORMNR       102360 non-null  int64  
 7   KON          102360 non-null  int64  
 8   FORMID       102360 non-null  int64  
 9   ALD00        102360 non-null  int64  
 10  ALD01        102360 non-null  int64  
 11  ALD03        102360 non-null  int64  
 12  ALD03_1      102360 non-null  int64  
 13  ALD05        102360 non-null  int64  
 14  ALD05_2      102360 non-null  int64  
 15  ALD10        102360 non-null  int64  
 16  ALD15        102360 non-null  int64  
 17  ALD15_1      102360 non-null  int64  
 18  ALD15_2      102360 non-

Checking the names of all columns in the data

In [228]:
columns = censusSweden.columns

Calling the data only with specific columns

In [229]:
censusSweden = pd.read_csv(census_path, sep=';', usecols=[
                           'LANGENNMN'  # Standard name of the county for the geographical area in plain text
                           , 'GEOIDNMN'  # Standard name of the geographical area in plain text, i.e. not a source name
                           , 'GEOIDTYP'  # Type of breakdown of the geographical area  0 =Assembly, 1 = Pastorate, 2 = Other type, 3 = Several parishes, 9 = Part of a parish
                           , 'AR'  # Year
                           , 'KON'  # 1 = Man  2 = Female. I choose 1 but it could be 2 for the total population
                           , 'BEF_TOT'  # Total population at source
                           , 'BEF_GENTOT'  # Total population, generated
                           ])


Processing the census data such that corresponds only to Scania and the first population size registred for each parish.

In [256]:
censusScania = censusSweden.loc[((censusSweden['LANGENNMN'] == 'KRISTIANSTADS LÄN') | (
    censusSweden['LANGENNMN'] == 'MALMÖHUS LÄN')) & (censusSweden['KON'] == 1)]
censusScania.iloc[100:150]

,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
41004,KRISTIANSTADS LÄN,STENESTAD,0,1795,1,514,514
41006,KRISTIANSTADS LÄN,STENESTAD,0,1800,1,518,518
41008,KRISTIANSTADS LÄN,STENESTAD,0,1805,1,510,510
41010,KRISTIANSTADS LÄN,STENESTAD,0,1810,1,558,558
41012,KRISTIANSTADS LÄN,STENESTAD,0,1815,1,579,579
41014,KRISTIANSTADS LÄN,STENESTAD,0,1820,1,603,603
41016,KRISTIANSTADS LÄN,STENESTAD,0,1825,1,628,628
41018,KRISTIANSTADS LÄN,STENESTAD,0,1830,1,645,645
41020,KRISTIANSTADS LÄN,STENESTAD,0,1835,1,613,613
41022,KRISTIANSTADS LÄN,STENESTAD,0,1840,1,619,619


Cleaning the data: First, we remove given strings and white spaces at the end of a word. To do so, we must provide the string list to delete. In this step, you can use regular expressions.

In [277]:
import re

def replace_strings_and_regex(dataframe, column, patterns):
    for pattern in patterns:
        dataframe[column] = dataframe[column].apply(lambda x: re.sub(pattern, '', x))
    dataframe[column] = dataframe[column].str.strip()
    return dataframe


In [276]:
# Regex to delete the following strings:
# ', DEL (KRISTIANSTAD)', ', DEL (MALMÖHUS)', ', DEL (MALMÖHUS LÄN)'
# ,', DEL AV (FROSTA HÄRAD, MALMÖHUS LÄN)', ', DEL (EVERÖD, MALMÖHUS)'
# ,' DEL (HYLLINGE, MALMÖHUS)', ' (MALMÖ SF)', '(STAFFANSTORP)'
# ,' GARNISONSFÖRS.', ' OCH GARNISONSFÖRS.', ' STADS', ' STAD'

regex = r'(,?\s+DEL\s(AV\s)?\((\w+,?\s?)+\))|(\s*\((\w+,?\s?)+\))|(\s+(OCH\s)?GARNISONSFÖRS\.?)|(\s+STADS?)'

censusScania = replace_strings_and_regex(censusScania, 'GEOIDNMN', [
    'PASTORAT', 'HOSPITAL', ' LANDS', ' SLOTTSFÖRSAMLING', ' DOMKYRKOFÖRSAMLING', regex
])
censusScania.iloc[100:150]


,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
41004,KRISTIANSTADS LÄN,STENESTAD,0,1795,1,514,514
41006,KRISTIANSTADS LÄN,STENESTAD,0,1800,1,518,518
41008,KRISTIANSTADS LÄN,STENESTAD,0,1805,1,510,510
41010,KRISTIANSTADS LÄN,STENESTAD,0,1810,1,558,558
41012,KRISTIANSTADS LÄN,STENESTAD,0,1815,1,579,579
41014,KRISTIANSTADS LÄN,STENESTAD,0,1820,1,603,603
41016,KRISTIANSTADS LÄN,STENESTAD,0,1825,1,628,628
41018,KRISTIANSTADS LÄN,STENESTAD,0,1830,1,645,645
41020,KRISTIANSTADS LÄN,STENESTAD,0,1835,1,613,613
41022,KRISTIANSTADS LÄN,STENESTAD,0,1840,1,619,619


Process the data from Scania only to keep the first population size registered for each parish.
This was done following two approaches. 

1. First approach: We group the data by parish name and then select the minimum year. As the minimum year is not unique after deleted strings, this approach allows repetitions.

In [278]:
def process_dataframe_rep(df, groupby_column, year_column):
    # Group a Pandas DataFrame by a column
    parish_grp = df.groupby([groupby_column])
    # Get the unique values of a column as a list
    parish_grp_name = parish_grp[groupby_column].unique().tolist()

    result_df = pd.DataFrame()

    for name in parish_grp_name:
        grp_name = parish_grp.get_group(name[0])
        result_df = pd.concat(
            [result_df, (grp_name[grp_name[year_column] == grp_name[year_column].min()])], axis=0)

    return result_df

In [292]:
# Usage example with the 'censusScania' dataframe
popSizeScania = process_dataframe_rep(censusScania, 'GEOIDNMN', 'AR')
popSizeScania.shape

(405, 7)

In [293]:
popSizeScania.groupby(['GEOIDNMN']).get_group('HELSINGBORGS')
#popSizeScania.groupby(['GEOIDNMN']).size().to_csv('popSizeScania.csv', encoding='utf-8', index=True, header=True)

,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
57393,MALMÖHUS LÄN,HELSINGBORGS,0,1775,1,1290,1290
57426,MALMÖHUS LÄN,HELSINGBORGS,0,1775,1,453,453


2. Second approach: This method explores the given DataFrame exhaustively and keeps the required information in a dictionary. In our case, this information corresponds to the position associated with each parish name and the minimum year, according to the original DataFrame. This approach doesn't allow repetitions since the condition for replacing the information in the dictionary is strict (<).

In [294]:
# %%timeit
def process_dataframe(df, column1:str, column2:str):
    aux_dict = {}
    for i in range(len(df)):
        name_i = df[column1].iloc[i]
        ar_i = df[column2].iloc[i]
        if name_i in aux_dict:
            if ar_i < aux_dict[name_i]['min']:
                aux_dict[name_i] = {'min': ar_i, 'position': i}
        else:
            aux_dict[name_i] = {'min': ar_i, 'position': i}
    final_positions = [value['position'] for key, value in aux_dict.items()]
    return df.iloc[final_positions]


In [295]:
# Usage example with the 'censusScania' dataframe
popSizeScania = process_dataframe(censusScania, 'GEOIDNMN', 'AR')
popSizeScania.shape

(403, 7)

In [296]:
popSizeScania.groupby(['GEOIDNMN']).get_group('HELSINGBORGS')

,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
57393,MALMÖHUS LÄN,HELSINGBORGS,0,1775,1,1290,1290


In [238]:
def check_name(data: pd.DataFrame, heading: str, name: str):
    filt_name = data[heading].str.contains(name, na=False)
    return data.loc[filt_name]

In [301]:
from Levenshtein import distance as levenshtein_distance

def find_similar_names(data: pd.DataFrame, heading: str, name: str, threshold=0.8):
    aux_dict = {}
    for i in range(len(data)):
        name_i = data[heading].iloc[i]
        max_len_i = max(len(name), len(name_i))
        if max_len_i == 0:
            continue
        if (max_len_i - levenshtein_distance(name, name_i)) / max_len_i >= threshold:
            aux_dict[name_i] = {'distance': levenshtein_distance(name, name_i), 'position': i}
    final_positions = [value['position'] for key, value in aux_dict.items()]
    return data.iloc[final_positions]    
    

In [305]:
find_similar_names(popSizeScania, 'GEOIDNMN', 'HELSINGBOR')

,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
57393,MALMÖHUS LÄN,HELSINGBORGS,0,1775,1,1290,1290


In [240]:
def filter_data_by_name(data: pd.DataFrame, heading: str, input_names: list[str]):
    output_names = []
    for name in input_names:
        filter_data = check_name(data, heading, name)
        if len(filter_data) == 0 :
            output_names  = output_names  + [name]
        else:
            continue
    return output_names

In [241]:
popSizeScania_names = get_Names(popSizeScania, 'GEOIDNMN')
check_name(popSizeScania, 'GEOIDNMN', 'TJÖRNARP')

,LANGENNMN,GEOIDNMN,GEOIDTYP,AR,KON,BEF_TOT,BEF_GENTOT
44416,KRISTIANSTADS LÄN,TJÖRNARP,0,1775,1,456,456
44426,KRISTIANSTADS LÄN,"TJÖRNARP, DEL AV (FROSTA HÄRAD, MALMÖHUS LÄN)",9,1810,1,92,92


Reading the geographical information by parish

In [242]:
# Get the current working directory
data_folder = "MapScaniaSweden"
parishScania_path = os.path.join(data_folder, "Parishes1720_1890.shp")

parishScaniaMap = gpd.read_file(parishScania_path)
selected_columns = ['G_NAME','GET_END_YE', 'geometry']
parishScaniaMap = parishScaniaMap[selected_columns]

In [243]:
parishScaniaMap = delete_strings(
    parishScaniaMap, ['FÖRSAMLING'
                      , 'L LÄN'
                      , 'S LÄN'
                      , 'M LÄN'
                      , 'HELIGA TREFALDIGHETS'
                      , ' LANDSFÖRSAMLING'
                      , ' STADS'
                      , ' LANDS'
                      ])

TypeError: delete_strings() missing 1 required positional argument: 'list_expr'

In [ ]:
#%%timeit
aux_dict = {}
parishMap = gpd.GeoDataFrame() 

for i in range(len(parishScaniaMap)):
    name_i = parishScaniaMap['G_NAME'].iloc[i]
    ar_i = parishScaniaMap['GET_END_YE'].iloc[i]
    if name_i in aux_dict:
        if ar_i < aux_dict[name_i]['min']:
            aux_dict[name_i] = {'min': ar_i, 'position': i}
    else:
        aux_dict[name_i] = {'min': ar_i, 'position': i}
final_positions = [value['position'] for key, value in aux_dict.items()]
parishMap = parishScaniaMap.iloc[final_positions]   
print(parishMap.shape)            
    

In [ ]:
parishMap['G_NAME'][parishMap['G_NAME'] == 'TRELLEBORGS']

In [ ]:
fig,ax = plt.subplots(figsize=(13,10))
parishMap.plot(ax=ax, column = "G_NAME", edgecolor='black', legend=False)
plt.xlabel('Meters')
plt.ylabel('Meters')
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 0.2))
plt.show()

In [ ]:
parishMap_names = get_Names(parishMap, 'G_NAME')

In [ ]:
filter_data_by_name(popSizeScania, 'GEOIDNMN', parishesScania_names)

['SANKT OLOF',
 'YSTAD SANKT PETRI',
 'YSTAD SANKT MARIA',
 'JERRESTAD',
 'SKÖRUP',
 'LILLA HARRIE ',
 'ÖRTOFTA ',
 'MALMÖ CAROLI',
 'MALMÖ GARNISONEN',
 'MALMÖ SANKT PETRI',
 'MELLANGREVIE',
 'SÄRLÖV',
 'VÄSTRA KÄRRSTORP ',
 'STORA SLÅGARP ',
 'TRELLEBORG ',
 'VELLINGE ',
 'VÄSTRA ALSTAD ',
 'VÄSTRA TOMMARP ',
 'VÄSTRA VEMMERLÖV ',
 'BONDERUP ',
 'FLÄDIE ',
 'NORRA NÖBBELOV',
 'HASSLE BÖSARP',
 'KÄLLSTORP ',
 'LILLA ISIE ',
 'SIMLINGE ',
 'SÖDRA ÅBY ',
 'ÄSPÖ ',
 'ÖSTRA KLAGSTORP ',
 'ÖSTRA TORP ',
 'SÖDRA RÖRUM ',
 'ÖSTRA SALLERUP ',
 'ÖSTRA STRÖ ',
 'ÖSTRA ÄSPINGE ',
 'LÅNGARÖD',
 'TOLÅNGA ',
 'BÅSTAD ',
 'HJÄRNARP ',
 'VÄSTRA KARUP ',
 'NORRA VRAM ',
 'VÄLLUV ',
 'VÄSBY ',
 'FÄRINGTOFTA ',
 'GRÅMANTORP',
 'MUNKA LJUNGBY ',
 'TÅSTARP ',
 'ÖRKELLJUNGA ',
 'ANNELÖV ',
 'KÄLLS NÖBBELÖV ',
 'NÄS – SE GULLARP ',
 'VÄSTRA STRÖ ',
 'ÖSTRA KARABY ',
 'SIREKÖPINGE ',
 'ST IBB',
 'TOFTA ',
 'HUARÖD ',
 'LINDERÖD ',
 'MAGLEHEM ',
 'ÄSPHULT ',
 'ÖSTRA SÖNNARSLÖV ',
 'IGNABERNA',
 'NORRA SANDBY'

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    print(southeastParishesNoPop)
    

In [ ]:
filter_data_by_name(parishScaniaMap, parishScaniaMap['G_NAME'], popSizeScania_names)

NameError: name 'filter_data_by_name' is not defined

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

In [ ]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    #print(southeastParishesNoPop)
    

Check regular expressions code in Mathematica for eliminATING SOME STRINGS

In [ ]:

parishesScaniaNoPop = []

for name in parishesScania_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        parishesScaniaNoPop  = parishesScaniaNoPop  + [name]
    else:
        continue

    

In [ ]:
check_name(popSizeScania, popSizeScania['GEOIDNMN'], 'YSTAD')
check_name(popSizeScania, popSizeScania['GEOIDNMN'], 'OLOFS')


In [ ]:
type(censusScania['GEOIDNMN'])

In [ ]:
filt2 = popSizeScania['GEOIDNMN'].str.contains('NORRA RÖRUM', na=False)
#popSizeScania.loc[filt2]

In [ ]:
print(popSizeScania[['GEOIDNMN','AR']].iloc[2])